## Problem definition

Our objective is to use reinforcement learning to learn a policy of routes for a trash collector truck. An optimized oath can help in obtaining a faster and more efficient trash collection. The problem can be modeled as identifying a path inside a graph. This problem have been previously solved using other optimization approachs, heuristics and graph algorithms, however we want to apply reinforcement learning in it for educational purposes and compare it with other approaches. The problem can be defined as follow:

**Def. 1**: With a weighted directed graph $G = (V, E)$, and a starting node $v$, find the shortest path to node $u$.

**Def. 2**: With a weighted directed graph $G = (V, E)$, and a set of nodes $U = \{u_1, \dots, u_k\}$, find the shortest path that traverse all nodes.

The second definition is a harder problem in which the model has to learn the best order of collection. The problem can be modeled in the reinforcement learning format as follow:

**Reinforcement learning model**:
- Episodic, finishes when arrive on destination node.
- States: $n$ states $v_i \in V$ , each is a node of the graph.
- Actions: $n$ action, each one correspond to moving to a specific node. It is important to note that the action to move to node $u$, if the state is node $v$, can only be made if there is an edge from $v$ to $u$.
- Reward will be $0$ at any node and will be $1$ if the node is the desired one (or one of the set of desired nodes). Impossible actions (moving trought a non-existent edge) will have reward equal to -1.

In [79]:
import numpy as np
import networkx as nx
from tqdm import tqdm

### Q-learning solution

https://www.datacamp.com/tutorial/introduction-q-learning-beginner-tutorial

https://www.deeplearningbook.com.br/algoritmo-de-agente-baseado-em-ia-com-reinforcement-learning-q-learning/


$$Q^{\pi} (s_t, a_t) = \mathbb{E} \left[ R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3}+ \dots | s_t, a_t\right]$$

$$Q (s_t, a_t) \leftarrow Q (s_t, a_t) + \delta \left[ R_{t+1} + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t)\right] $$

In [94]:
class Environment:
    """
    Deterministic simulation that returns the state based in the action.
    It also returns the reward in the following format:
        -1: if the action is not valid
        0: if the action is valid but the destination is not reached
        1: if the action is valid and the destination is reached
    """
    def __init__(self, G, source, dest, reward = "unit"):
        self.G = G
        self.source = source
        self.dest = dest
        self.reward = reward

    def reset(self):
        self.state = self.source
        return self.state
    
    def step(self, action):
        """Return new state, reward, and if the destination is reached"""
        # action is a node, verify if it is a neighbor
        # if is not a neighbor, stay in the same state
        if self.reward == "unit":
            neighbors = list(self.G.neighbors(self.state))
            if action not in neighbors:
                return self.state, -1, False
            
            self.state = action
            if self.state == self.dest:
                return self.state, 1, True
            else:
                return self.state, 0, False
        elif self.reward == "weighted":
            neighbors = list(self.G.neighbors(self.state))
            if action not in neighbors:
                return self.state, -100, False
            
            if action == self.dest:
                self.state = action
                return self.state, 100, True
            else:
                w = self.G[self.state][action]['weight']
                self.state = action
                return self.state, -w, False
    

In [81]:
def greedy_policy(Q, state):
    """Greedy policy that returns the action with the highest Q value"""
    return np.argmax(Q[state, :])

def epsilon_greedy_policy(Q, state, epsilon):
    """Epsilon greedy policy that returns a random action with probability epsilon"""
    if np.random.uniform(0, 1) < epsilon:
        return np.random.randint(len(Q[state, :]))
    else:
        return greedy_policy(Q, state)

def train_q_learning(
    Q,
    learning_rate = 0.7,
    gamma = 0.95,
    min_epsilon = 0.05,
    max_epsilon = 1,
    n_episodes = 10000,
    max_steps = 100,
):
    """Training of the Q table using Q learning algorithm. 
    It needs the enviroment env and the Q table.

    :param Q: numpy array with the Q table
    :param learning_rate: learning rate of algorithm, defaults to 0.7
    :param gamma: weight of future rewards, defaults to 0.95
    :param min_epsilon: min probability of exploration, defaults to 0.05
    :param max_epsilon: max probability of exploration, defaults to 1
    :param n_episodes: number of episodes, defaults to 10000
    :param max_steps: max number of steps per episode, defaults to 100
    """
    epsilon = max_epsilon
    rechead_dest = False
    
    for episode in tqdm(range(n_episodes)):
        # Start episode again resetting the enviroment
        state = env.reset()

        for step in range(max_steps):
            # Choose action and get reward
            action = epsilon_greedy_policy(Q, state, epsilon)
            new_state, reward, done = env.step(action)
            
            # update Q table based on Bellman equation
            Q[state, action] += learning_rate * (reward + gamma * np.max(Q[new_state, :]) - Q[state, action])
            state = new_state
            if done and not rechead_dest:
                rechead_dest = True
                print(f"Episode {episode} reached destination in {step} steps")
        
            if done:
                break
        
        
        epsilon -= (max_epsilon - min_epsilon) / n_episodes # make it decay exponentially


def test_q_learning(Q):
    state = env.reset()
    actions = [state]
    for _ in range(1000):
        action = greedy_policy(Q, state)
        state, _, done = env.step(action)
        actions.append(action)
        if done:
            break
        
    return actions

In [101]:
n_nodes = 1000
G = nx.barabasi_albert_graph(n_nodes, 3)
# if there is an edge between 0 and 99, remove it
if G.has_edge(0, n_nodes - 1):
    G.remove_edge(0, n_nodes - 1)
# add random weights to edges
for edge in G.edges():
    G.edges[edge]["weight"] = np.random.randint(1, 10)
print("Path between first and last node:", nx.shortest_path(G, 0, n_nodes - 1))
# print weight of the shortest path
print("Weight of the shortest path:", nx.shortest_path_length(G, 0, n_nodes - 1, weight = "weight"))



if n_nodes < 200:
    nx.draw(G)
env = Environment(G, 0, n_nodes - 1, reward = "weighted")

Path between first and last node: [0, 6, 320, 999]
Weight of the shortest path: 11


In [104]:
Q = np.zeros((len(G.nodes), len(G.nodes)))
train_q_learning(Q, min_epsilon=0.1)

 16%|█▋        | 1637/10000 [00:01<00:05, 1404.13it/s]

Episode 1334 reached destination in 84 steps


100%|██████████| 10000/10000 [00:02<00:00, 4080.88it/s]


In [105]:
test_q_learning(Q)

[0, 413, 406, 999]

In [106]:
path = [0, 413, 406, 999]
lenght = 0
for i in range(len(path) - 1):
    lenght += G[path[i]][path[i + 1]]['weight']
print("Lenght of the path:", lenght)

Lenght of the path: 16
